# Dados sobre os clubes nesse período da Libertadores.

In [5]:
import pandas as pd


In [6]:
df = pd.read_csv("libertadores-results-ds.csv")
df.head()

,Edition,Round,Date,Home Club,Away Club,Home Score,AwayScore
0,2023,Final,4/11/2023,Fluminense FC,Boca Juniors,2,1
1,2023,Semifinal,6/10/2023,Palmeiras,Boca Juniors,1,1
2,2023,Semifinal,5/10/2023,Internacional,Fluminense FC,1,2
3,2023,Semifinal,29/09/2023,Boca Juniors,Palmeiras,0,0
4,2023,Semifinal,28/09/2023,Fluminense FC,Internacional,2,2


# Vendo a quantidade total de jogos do dataset

In [7]:
len(df)

1954

# Vendo a quantidade diferente de times no data set

In [8]:
# Contagem de times diferentes no dataset
# Um time joga no MINIMO 1 jogo em casa, por isso pode usar essa coluna
df['Home Club'].nunique()


132

# Vendo times com mais jogos

In [9]:
df['Home Club'].value_counts()

Home Club
River Plate            62
Boca Juniors           57
Nacional               54
Palmeiras              54
Atlético Mineiro       54
                       ..
Deportivo Petare        1
Everton CD              1
Deportivo Capiatá       1
Club Puebla             1
Sportivo Trinidense     1
Name: count, Length: 132, dtype: int64

# Colunas

In [11]:
df.columns

Index(['Edition', 'Round', 'Date', 'Home Club', 'Away Club', 'Home Score',
       'AwayScore'],
      dtype='object')